# preprocessing data

In [1]:

#!/usr/bin/env python
#-*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import time
import requests  # pip install requests
import json
import re
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import xml.etree.ElementTree as ET
import datetime
import pymssql
import numpy as np

#remove accents vietnamese
def remove_accents(text):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
#     s1.encode('utf-8')
#     s0.encode('utf-8')
    if text == np.nan or text == np.NAN or text == np.NaN:
        return text
    else:    
        s = ''
        text = str(text)
        text = text.decode('utf-8')

        for c in text:
            if c in s1:
                s += s0[s1.index(c)]
            else:
                s += c
        return s.strip().replace(" ","_").replace(",", "-")

    
# logging
def log():
    import logging
    logging.basicConfig(filename='/home/tgdd/TA_69/recommenderr/log.recommendation/log_vuivui.txt',
                                filemode='a',
                                format = "%(asctime) -10s, %(levelname)s, %(module)s:%(lineno)s, %(funcName)s, %(message)s",
                                level=logging.ERROR)
    logger = logging.getLogger()
    return logger

log = log()

In [2]:
#connect to our cluster
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': '172.16.3.105', 'port': 9200}])
# res = requests.get('http://172.16.3.105:9200/index_product/product/86734')
# print(res.content)

# get list category ID by list product ID
def get_list_cate_ID(list_product_ID):  
    global list_cate_ID
    list_cate_ID = []
#     f = open('product_cate.txt', 'wb')
    for productID in list_product_ID:
        try:
            pathCate = es.get(index="index_product", doc_type="product", id=str(productID))['_source']['categoryPath']
            listCate = pathCate.split(" ")
#             f.write(str(productID) +','+listCate[-1]+"\n")
            list_cate_ID.append(listCate[-1])
        except Exception as error:
            log.exception(str(error))
#     f.close()
    return list_cate_ID


def get_cate_ID_by_proID_position(productID, position, es):
    try:
        #connect to our cluster
#         from elasticsearch import Elasticsearch
#         es = Elasticsearch([{'host': '172.16.3.105', 'port': 9200}])
        # res = requests.get('http://172.16.3.105:9200/index_product/product/86734')
        # print(res.content)
        pathCate = es.get(index="index_product", doc_type="product", id=str(productID))['_source']['categoryPath']
        listCate = pathCate.split(" ")
        return int(listCate[position])
    except Exception as error:
        log.exception(str(error))   
        return np.nan

def get_cate_ID_by_proID(productID):
    try:
        pathCate = es.get(index="index_product", doc_type="product", id=str(productID))['_source']['categoryPath']
        listCate = pathCate.split(" ")
        return int(listCate[-1])
    except Exception as error:
        return np.nan
    

def get_name_cate_by_cate_ID(cate_ID):
    try: 
        url = 'http://webservice.vuivui.com/productsvc.asmx/GetProductCategoryByCate?intSiteID=8&intCategoryID='
        url = url + str(int(cate_ID))
        res = requests.get(url)
        xml = ET.fromstring(res.content)
        return xml[0][1].text
    except Exception as error:
        log.exception(str(error))
        return ""
    
#     get list name cate by list cate ID
def get_list_name_cate(list_cate_ID, filename = 'cate_name.txt'):
    f = open(filename, 'wb')
    for cate_ID in list_cate_ID:
        f.write(cate_ID + ',' + get_name_cate_by_cate_ID(cate_ID) + '\n')
    f.close()    

In [3]:
# ////////////////////////

In [4]:
def read_all_csv_into_df(path = r'/home/tgdd/TA_69/vuivui/data_vuivui'):
    import glob
#     path =r'/home/tgdd/TA_69/vuivui/data_vuivui' # use your path
    allFiles = glob.glob(path + "/*.csv")
    frame = pd.DataFrame()
    try:
        for file_ in allFiles:
            df = pd.read_csv(file_, header=None)
            frame = frame.append(df)
    except:
        pass
    return frame

def read_df_fromfile():
    # read saleoder data,'/home/tgdd/TA_69/vuivui/20170614.csv'
    
    data_file =read_all_csv_into_df()
    data_file.columns = ['SaleOrderID','ProductID','Quantity','StandardSalePrice']

    saleorderDF = data_file[['SaleOrderID','ProductID','Quantity']]
    # data1['CateID'] = data1["ProductID"].apply(get_cate_ID_by_proID)
    # data1['CateID_'] = data1['CateID']
    # data1['ProductID_']=data1['ProductID']
    # data1['NameCate'] = data1['CateID'].apply(get_name_cate_by_cate_ID).apply(remove_accents)

    list_product_ID = list(set(saleorderDF.ProductID))
    df = pd.DataFrame(list_product_ID, columns=['ProductID'])
    df['CateID'] = df.ProductID.apply(get_cate_ID_by_proID)
    df['CateID_'] = df['CateID']
    df = df.dropna(how = 'any', subset=['CateID'])
    df = df.drop_duplicates()
    df = df[~df.CateID.isin(['3103', '2102'])]
    # df['NameCate']= df.CateID.apply(get_name_cate_by_cate_ID)
    # df['NameCate_']=df['NameCate'].apply(remove_accents)
    saleorderDF = saleorderDF.merge(df, how='left', on = 'ProductID')
    # data1['NameCate_2']=data1['NameCate_']
    return saleorderDF


def convert_format_csv(pivot_DF, filename):
    pivot_DF.to_csv('big_df_tmp.txt',header=False, index=False)#write dataframe to text file, separate = ","
    f1 = open(filename, 'wb')#read file big_df.txt then write into sparse_df.txt, sparse dataframe
    f = open('big_df_tmp.txt', 'rb')
    for line in f:
        line = ' '.join(line.split(','))
        a = ','.join(line.split())
        if(len(a)>0):
            f1.write(a+'\n')
    f1.close()
    f.close()


In [5]:
# # data2.columns = data2.columns.droplevel(0) #remove amount
# data2.columns.name = None               #remove categories
# data2 = data2.reset_index() 

# start = time.time()
# # write dataframe to text file, it not good due to run slowlly (shouldn't use)
# f = open("df_to_text.txt", 'wb')
# for i in range(data2.shape[0]):
#     for j in range(data2.shape[1]):
#         if(data2.iloc[i][j] > 0):
#             f.write(str(data2.columns[j]) + ',')
#     f.write('\n')
# f.close()

# duration = time.time() - start            

# Connect to Redis

In [6]:


def connect_to_redis():
    # pip install redis-py-cluster
    import redis
    from rediscluster import StrictRedisCluster
    # Requires at least one node for cluster discovery. Multiple nodes is recommended.
#     startup_nodes = [{"host": "192.168.2.190", "port": "6379"}]
    startup_nodes = [{"host": "10.1.12.30", "port": "16379"}]
    rc = StrictRedisCluster(startup_nodes=startup_nodes, decode_responses=True)
    return rc
    #!redis-cli -h 10.1.12.30 -p 16379 -c
    #rc.dbsize()

def sethash(hashname, tmp2, rc):
    for index, row in tmp2.iterrows():
        rc.hset(hashname,str(row.A).replace(' ', ''), str(row.B).replace(' ', ''))

def getall(hashname, rc):
    return rc.hgetall(hashname)
#     def deletehash(self, hashname):
#         if



# get automatically saleorderID

In [7]:
#! /usr/bin/env python
# need to install freeSSHd.exe and config on remote machine

'''
it will run file webservices.exe from 10.1.12.111\C:\Release\webservices.exe
result file save at 10.1.12.111\C:\Program Files (x86)\freeSSHd
with name is the date that it run
then it will be copy result file to 10.1.4.60
'''
import paramiko
import sys
import time

SERVER = '10.1.12.111'
USER = 'tgdd'
PASSWORD = 'Tgdd2012'
port = 8896

def up_file():
    try:
        paramiko.util.log_to_file('paramiko.log')
        transport = paramiko.Transport((SERVER, port))
        transport.connect(username=USER, password=PASSWORD)
        sftp = paramiko.SFTPClient.from_transport(transport)

        sftp.chdir('')
        filename = time.strftime("%Y%m%d")+'.csv'#get today
        filepath = './'+filename
        localpath = '/home/tgdd/TA_69/vuivui/data_vuivui/'+filename
        sftp.get(filepath, localpath)
    #         filepath = '/home/zeth/lenna.jpg'
    #         localpath = '/home/zeth/lenna.jpg'
    #         sftp.put(filepath, localpath)
    finally:
        sftp.close()
        transport.close()


class MySSHClient():
    def __init__(self, domain=SERVER, username=USER, password=PASSWORD, port = port):
        self.domain = domain
        self.username = username
        self.password = password
        self.port = port
        self.connection = None
        self.return_str = ''
        self.has_error = False

    def do_connect(self):
        self.connection = paramiko.SSHClient()
        self.connection.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        self.connection.connect(self.domain, username=self.username, password=self.password, port = self.port)

    def execute_command(self, command):
        if command:
            stdin, stdout, stderr = self.connection.exec_command(command)
            stdin.close()
            error = str(stderr.read())
            if error:
                self.has_error = True
                self.return_str = error
                print 'has error'
            else:
                # reset has_error to the default value
                self.has_error = False
                self.return_str = str(stdout.read())
                print 'no error'

            print self.return_str
        else:
            print "no command to execute"

    def do_close(self):
        self.connection.close()
      
        
# if __name__ == '__main__':
#     client = MySSHClient()
#     client.do_connect()
#     today = datetime.date.today()
#     today = today.strftime('%d/%m/%Y')

# #     command = sys.argv[1]
# #     command = "cmd /c cd D:\Data && D: && fibonaci.exe"
#     command = 'cmd.exe /c echo start C:\Release\\webservices.exe '+today+' '+ today +'> C:\Release\\runme.bat && C:\Release\\runme.bat'
# #     command = 'cmd /c C:\test\Debug\test.exe'

#     client.execute_command(command)
#     client.do_close()
# # turn off interactive services detection

# Apriori algorithm

In [8]:
"""
Usage:
    $python apriori.py -f DATASET.csv -s minSupport  -c minConfidence
    $python apriori.py -f DATASET.csv -s 0.15 -c 0.6
"""
import sys
import time
from itertools import chain, combinations
from collections import defaultdict
from optparse import OptionParser
from threading import Thread
import time

class Apriori:
    minSupport = 0
    minConfidence = 0
    def __init__(self, Support, Confidence):
        self.minSupport = Support
        self.minConfidence = Confidence
        
    def subsets(self,arr):
        """ Returns non empty subsets of arr"""
        return chain(*[combinations(arr, i + 1) for i, a in enumerate(arr)])

    def returnItemsWithMinSupport(self,itemSet, transactionList, minSupport, freqSet):
            """calculates the support for items in the itemSet and returns a subset
           of the itemSet each of whose elements satisfies the minimum support"""
            _itemSet = set()
            localSet = defaultdict(int)

            for item in itemSet:
                    for transaction in transactionList:
                            if item.issubset(transaction):
                                    freqSet[item] += 1
                                    localSet[item] += 1

            for item, count in localSet.items():
                    support = float(count)/len(transactionList)

                    if support >= minSupport:#>=
                            _itemSet.add(item)

            return _itemSet


    def joinSet(self,itemSet, length):
            """Join a set with itself and returns the n-element itemsets"""
            return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


    def getItemSetTransactionList(self,data_iterator):
        transactionList = list()
        itemSet = set()
        for record in data_iterator:
            transaction = frozenset(record)
            transactionList.append(transaction)
            for item in transaction:
                itemSet.add(frozenset([item]))              # Generate 1-itemSets
        return itemSet, transactionList


    def runApriori(self,data_iter, minSupport, minConfidence):
        """
        run the apriori algorithm. data_iter is a record iterator
        Return both:
         - items (tuple, support)
         - rules ((pretuple, posttuple), confidence)
        """
        itemSet, transactionList = self.getItemSetTransactionList(data_iter)

        freqSet = defaultdict(int)
        largeSet = dict()
        # Global dictionary which stores (key=n-itemSets,value=support)
        # which satisfy minSupport

        assocRules = dict()
        # Dictionary which stores Association Rules

        oneCSet = self.returnItemsWithMinSupport(itemSet,
                                            transactionList,
                                            minSupport,
                                            freqSet)

        currentLSet = oneCSet
        k = 2
        while(currentLSet != set([])):
            largeSet[k-1] = currentLSet
            currentLSet = self.joinSet(currentLSet, k)
            currentCSet = self.returnItemsWithMinSupport(currentLSet,
                                                    transactionList,
                                                    minSupport,
                                                    freqSet)
            currentLSet = currentCSet
            k = k + 1

        def getSupport(item):
                """local function which Returns the support of an item"""
                return float(freqSet[item])/len(transactionList)

        toRetItems = []
        for key, value in largeSet.items():
            toRetItems.extend([(tuple(item), getSupport(item))
                               for item in value])

        toRetRules = []
        for key, value in largeSet.items()[1:]:
            for item in value:
                _subsets = map(frozenset, [x for x in self.subsets(item)])
                for element in _subsets:
                    remain = item.difference(element)
                    if len(remain) > 0:
                        confidence = getSupport(item)/getSupport(element)
                        if confidence >= minConfidence:
                            toRetRules.append(((tuple(element), tuple(remain)),
                                               confidence))
        return toRetItems, toRetRules


    def printResults(self,items, rules):
        """prints the generated itemsets sorted by support and the confidence rules sorted by confidence"""
        for item, support in sorted(items, key=lambda (item, support): support):
            print "item: %s , %.3f" % (str(item), support)
        print "\n------------------------ RULES:"
        for rule, confidence in sorted(rules, key=lambda (rule, confidence): confidence):
            pre, post = rule
            print "Rule: %s ==> %s , %.3f" % (str(pre), str(post), confidence)
    def saveResults(self,items, rules):
        """save the generated itemsets sorted by support and the confidence rules sorted by confidence"""
        f = open('results.txt', 'wb')
        for item, support in sorted(items, key=lambda (item, support): support):
            f.write("\nitem: %s , %.3f" % (str(item), support)) 
        f.write("\n------------------------ RULES:")
        i = 0
        for rule, confidence in sorted(rules, key=lambda (rule, confidence): confidence):
            pre, post = rule
            i+=1
            f.write("\nRule %s : %s ==> %s , %.3f" % (str(i), str(pre), str(post), confidence)) 

    def dataFromFile(self,fname):
            """Function which reads from the file and yields a generator"""
            file_iter = open(fname, 'rU')
            for line in file_iter:
                    line = line.strip().rstrip(',')                         # Remove trailing comma
                    record = frozenset(line.split(','))
                    yield record

        
def run_apriori(saleorderDF):
    pivot_saleorderDF = saleorderDF.pivot_table('CateID', index = 'SaleOrderID',columns='CateID_',aggfunc=lambda x: ' '.join(str(v) for v in x))
    convert_format_csv(pivot_saleorderDF, 'sparse_df.txt')
    
    test = Apriori(0.0005, 0.5)
    inFile = test.dataFromFile('sparse_df.txt')
    items, rules = test.runApriori(inFile,test.minSupport,test.minConfidence)
    test.saveResults(items, rules)
    
    tmp = pd.DataFrame(rules)
    tmp1 = pd.DataFrame(list((tmp[0].values)))
    tmp1.columns = ['A', 'B']
    tmp2 = pd.pivot_table(tmp1, index='A', aggfunc=lambda x: ' '.join(str(v) for v in x))
    tmp2 = tmp2.reset_index()
    tmp2.B = tmp2['B'].apply(lambda row: [int(float(x)) for x in re.findall(r"\d+\.?\d*", str(row))])
    #     add
    tmp2.B = tmp2['B'].apply(lambda row: list(set(row)))
    tmp2.A = tmp2['A'].apply(lambda row: sorted(list(int(float(x)) for x in row)))
    return tmp2

def thread_run_apriori(threadname):
    while 1:
        try:
            tmp2 = run_apriori(saleorderDF)
            rc = connect_to_redis()
            if rc.exists('RecomSys_tmp') and tmp2.shape[0] > 100:
                rc.delete('RecomSys_tmp')
            sethash('RecomSys_tmp', tmp2, rc)
            rc.expire('RecomSys_tmp', 5184000)#set a key's time to live in seconds
            log.error('\n===================thread_run_apriori is working...======================')
            time.sleep(2*86400)
        except Exception as error:
            log.exception(str(error))
            

def pull_data_from_server():
    global saleorderDF
    client = MySSHClient()
    client.do_connect()
    today = datetime.date.today()
    start_date = today - datetime.timedelta(days=2)
    today = today.strftime('%d/%m/%Y')
    start_date = start_date.strftime('%d/%m/%Y')
#     command = sys.argv[1]
#     command = "cmd /c cd D:\Data && D: && fibonaci.exe"
    command = 'cmd.exe /c echo start C:\Release\\webservices.exe '+start_date+' '+ today +'> C:\Release\\runme.bat && C:\Release\\runme.bat'
#     command = 'cmd /c C:\test\Debug\test.exe'

    client.execute_command(command)
    client.do_close()
# turn off interactive services detection
    time.sleep(200)#set time to sleep in seconds to wait the job pull data finish
    up_file()
    time.sleep(10)
    saleorderDF = read_df_fromfile()


def thread_pull_data(threadname):
    while 1:
        pull_data_from_server()
        log.error('\n======================thread_pull_data is working...=========================')
        time.sleep(2*86400)
        
    
        
if __name__ == "__main__":
#     read all csv files then preprocessing
    saleorderDF = 0
    thread_pull_data = Thread( target = thread_pull_data, args =('Thread pull data from server', ))
    thread_pull_data.start()
    time.sleep(50)
#     run apriori, get result to store into redis db
    thread_run_apriori = Thread( target=thread_run_apriori, args=("Thread run apriori algorithm", ) )
    thread_run_apriori.start()
    



# run association rule for each user

In [9]:
# a9d84f26-0e54-4c24-b40e-1f6713987677
# 2b05443b-ab18-4437-9090-c434aba3d7da
def connectDB():
    server = "10.1.12.197"
    user = "rd"
    password = "rd123456"
    conn = pymssql.connect(server, user, password, "RD")
    cursor = conn.cursor(as_dict = True)
    return cursor

def get_basket_customer():
    cursor = connectDB()
#     basket_customer = "use RD SELECT ID, SESSION, PRODUCTID, NUMBERITEM FROM [RD].[dbo].[TRACKINGDATA] \
#     where SITEID = 8 and PRODUCTID != -1 and [COOKIE] = {0} order by ID desc".format("'"+user_cookie+"'", 'utf8')
    basket_customer = "use RD SELECT ID, COOKIE, CREATEDTIME,  SESSION, PRODUCTID, NUMBERITEM FROM [RD].[dbo].[TRACKINGDATA] \
    where SITEID = 8 and PRODUCTID != -1 order by ID desc"
    cursor.execute(basket_customer)
    basket_customer = cursor.fetchall()
    basket_customer = pd.DataFrame(basket_customer)
    cursor.close()
    basket_customer['CREATEDDAY'] = basket_customer.CREATEDTIME.apply(lambda x: x.to_pydatetime().strftime('%Y%m%d'))
    return basket_customer

def get_transaction(basket_customer, user_cookie):
    list_order_id = list()
    list_session = list()
    basket_market_one_user = basket_customer[basket_customer['COOKIE']== user_cookie]
    list_session = list(basket_market_one_user.SESSION.unique())
    # re.findall(r"\d+\.?\d*", str(row))
    list_day = list(basket_market_one_user.CREATEDDAY.unique())
#     if len(list_session) < 2 and len(list_day) > 1: #kiem tra xem khach chi dung 1 session duy nhat, de tach don hang
    for day in list_day:#loc theo ngay
        basket_customer_day = basket_market_one_user[basket_market_one_user['CREATEDDAY']==day]
        data = basket_customer_day.drop_duplicates(subset = ['PRODUCTID'])
        for item in list(data[data['NUMBERITEM']< 0].PRODUCTID):
            data.loc[data.PRODUCTID == item, 'NUMBERITEM'] = basket_customer_day[basket_customer_day['PRODUCTID']==item].NUMBERITEM.sum()
        record = data[data['NUMBERITEM']>0]
        if record.shape[0] > 0:
#                 list_order_id.append(list(record.PRODUCTID.unique()))
            record1 = frozenset(record.PRODUCTID)
            yield record1
# # ////
#     else:   
#         for session in list_session:#loc theo session
#             basket_customer_session = basket_market_one_user[basket_market_one_user['SESSION']==session]
#             data = basket_customer_session.drop_duplicates(subset = ['PRODUCTID'])
#             for item in list(data[data['NUMBERITEM']< 0].PRODUCTID):
#                 data.loc[data.PRODUCTID == item, 'NUMBERITEM'] = basket_customer_session[basket_customer_session['PRODUCTID']==item].NUMBERITEM.sum()
#             record = data[data['NUMBERITEM']>0]
#             if record.shape[0] > 0:
# #                 list_order_id.append(list(record.PRODUCTID.unique()))
#                 record1 = frozenset(record.PRODUCTID)
#                 yield record1
        
    
    
    #     write to file record       
#     with open("list_to_file1.txt", "w") as output:
#         for item in list_order_id:
#             output.write('%s\n' %','.join(map(str,item)))    

def recommend_for_one_user(basket_customer, user_cookie):
    # need to increase minsupport when enough data
    total_transaction = sum(1 for i in get_transaction(basket_customer, user_cookie))
    if total_transaction > 1:# nhieu hon 1 don hang
        if total_transaction < 10:
            minsupport = 2.0/total_transaction
        elif total_transaction < 30:    
            minsupport = 3.0/total_transaction #get_transaction is the total transaction
        elif total_transaction >= 30: 
            minsupport = 4.0/total_transaction

        test1 = Apriori(minsupport, 0.5)
        inFile = get_transaction(basket_customer,user_cookie)
        items, rules = test1.runApriori(inFile,test1.minSupport,test1.minConfidence)
        test1.saveResults(items, rules)

        tmp = pd.DataFrame(rules)
        if tmp.shape[0]> 0:
            tmp1 = pd.DataFrame(list((tmp[0].values)))
            tmp1.columns = ['A', 'B']
            tmp2 = pd.pivot_table(tmp1, index='A', aggfunc=lambda x: ' '.join(str(v) for v in x))
            tmp2 = tmp2.reset_index()
            tmp2.B = tmp2['B'].apply(lambda row: [int(float(x)) for x in re.findall(r"\d+\.?\d*", str(row))])
        #     add
            tmp2.B = tmp2['B'].apply(lambda row: list(set(row)))
            tmp2.A = tmp2['A'].apply(lambda row: sorted(list(int(float(x)) for x in row)))
        #     sort by lenght of values in columns A
            s = tmp2.A.str.len().sort_values().index
            tmp2=tmp2.reindex(s)
            # pd.DataFrame.reindex(s)
        #     just get the values have lenght equal 1
            tmp2 = tmp2[tmp2.A.str.len()==1]

            not_recommend = [2644, 2423, 2424, 4549, 3851, 4788]#dong ho, dien tu, dien lanh, dien gia dung, dien may, thiet bi van phong, dien thoai, lap top, linh kien, balo...
            for index_, raw in tmp2.iterrows():
                if get_cate_ID_by_proID_position(raw.A[0], 0, es) in not_recommend:
                    tmp2.drop(index_,  inplace = True)#delete the product in not_recommend
        else: 
            tmp2 = []
        return tmp2
    else:
        return []
    

# how to auto-adjust the minimum support threshold according the data size
# minsup = | x * e^(-ax - b) + c|



In [13]:
pull_data_from_server()

IOError: [Errno 2] No such file

In [13]:
basket_customer= pd.read_csv('basket_customer')

In [23]:
def get_transaction(basket_customer, user_cookie):
    list_order_id = list()
    list_session = list()
    basket_market_one_user = basket_customer[basket_customer['COOKIE']== user_cookie]
    list_session = list(basket_market_one_user.SESSION.unique())
    # re.findall(r"\d+\.?\d*", str(row))
    list_day = list(basket_market_one_user.CREATEDDAY.unique())
#     if len(list_session) < 2 and len(list_day) > 1: #kiem tra xem khach chi dung 1 session duy nhat, de tach don hang
    for day in list_day:#loc theo ngay
        basket_customer_day = basket_market_one_user[basket_market_one_user['CREATEDDAY']==day]
        data = basket_customer_day.drop_duplicates(subset = ['PRODUCTID'])
        for item in list(data[data['NUMBERITEM']< 0].PRODUCTID):
            data.loc[data.PRODUCTID == item, 'NUMBERITEM'] = basket_customer_day[basket_customer_day['PRODUCTID']==item].NUMBERITEM.sum()
        record = data[data['NUMBERITEM']>0]
        if record.shape[0] > 0:
            list_order_id.append(list(record.PRODUCTID.unique()))
    return list_order_id    
#             record1 = frozenset(record.PRODUCTID)
#             yield record1

In [27]:
user_cookie = 'a9d84f26-0e54-4c24-b40e-1f6713987677'
aaa = get_transaction(basket_customer, user_cookie)

In [28]:
aaa

[[88251, 81699, 77093, 79278, 76487, 81602, 90693, 81725, 78681, 82914, 85837],
 [111455, 99534, 76412, 80968, 74480, 108377, 95906, 111349, 90971],
 [100997,
  111059,
  74324,
  82902,
  85679,
  88482,
  110767,
  76339,
  76108,
  101070,
  81556,
  76560,
  79173,
  76450,
  90965,
  90966,
  90968,
  79138,
  79183,
  103404,
  87100,
  76444,
  79204,
  92962,
  102110],
 [103404, 95256, 86812, 81227, 95752, 93219, 79956],
 [109090,
  109356,
  83240,
  102583,
  110730,
  105778,
  83473,
  79171,
  101064,
  76993,
  76491,
  78254,
  88471,
  84818,
  79956,
  76108,
  109089,
  85164],
 [85679,
  76491,
  85808,
  79336,
  105838,
  110730,
  77169,
  79278,
  77107,
  79956,
  76328,
  84814,
  96629,
  89295,
  86167,
  76094,
  109089,
  92042,
  78886,
  93158,
  81563,
  90217,
  82593],
 [89400, 109089],
 [104540,
  90785,
  93205,
  95154,
  76449,
  93989,
  100721,
  79798,
  79594,
  90113,
  90111,
  85596,
  86152],
 [77750,
  106326,
  104764,
  76522,
  90965,


In [17]:
aaa.to_csv('aaa.csv')

AttributeError: 'generator' object has no attribute 'to_csv'

In [81]:
import math
X = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60,70 ]
def minsup(x, a, b, c):
    return x * math.exp(-a*x - b) + c

In [82]:
[minsup(x, .6, .2, .1) for x in X]

[0.30381101989183107,
 0.12029430636295735,
 0.1015155910275564,
 0.10010060911214223,
 0.10000626129093082,
 0.10000037407758358,
 0.10000002172826394,
 0.1000000012363275,
 0.10000000006924727,
 0.10000000000383069,
 0.1000000000000114,
 0.10000000000000003]

In [71]:
math.exp(100)

2.6881171418161356e+43

In [13]:
basket_customer = get_basket_customer()

In [105]:

def get_list_recommend_by_user(basket_customer):
    rc = connect_to_redis()
#     list_tmp = list()
    f = open('results_apriori.txt','wb')
    for cookie in basket_customer.COOKIE.unique():
        f.write(str(cookie)+'\n')
        df_tmp = recommend_for_one_user(basket_customer, cookie)
        if len(df_tmp)> 0:#neu co du lieu thi luu vao redis
#             if rc.exists(cookie):
#                 rc.delete(cookie)
#             sethash(cookie, df_tmp, rc)
#             rc.expire(cookie, 2592000)
#             list_tmp.append((cookie,df_tmp))
            f.write(str(df_tmp)+'\n')
    f.close()
    
        
    

In [106]:
get_list_recommend_by_user(basket_customer)


KeyboardInterrupt: 

In [10]:
cookie = '2a638266-82fe-4f88-940b-f3f8658b259e'
basket_customer[basket_customer['COOKIE'] == cookie].shape

NameError: name 'basket_customer' is not defined

In [121]:
user_cookie = '2a638266-82fe-4f88-940b-f3f8658b259e'

list_order_id = list()
list_session = list()
basket_market_one_user = basket_customer[basket_customer['COOKIE']== user_cookie]
list_session = list(basket_market_one_user.SESSION.unique())
# re.findall(r"\d+\.?\d*", str(row))
list_day = list(basket_market_one_user.CREATEDDAY.unique())
# if len(list_session) < 2 and len(list_day) > 1: #kiem tra xem khach chi dung 1 session duy nhat, de tach don hang
for day in list_day:#loc theo ngay
    basket_customer_day = basket_market_one_user[basket_market_one_user['CREATEDDAY']==day]
    data = basket_customer_day.drop_duplicates(subset = ['PRODUCTID'])
    for item in list(data[data['NUMBERITEM']< 0].PRODUCTID):
        data.loc[data.PRODUCTID == item, 'NUMBERITEM'] = basket_customer_day[basket_customer_day['PRODUCTID']==item].NUMBERITEM.sum()
    record = data[data['NUMBERITEM']>0]
    if record.shape[0] > 0:
            list_order_id.append(list(record.PRODUCTID.unique()))

# else:   
#     for session in list_session:#loc theo session
#         basket_customer_session = basket_market_one_user[basket_market_one_user['SESSION']==session]
#         data = basket_customer_session.drop_duplicates(subset = ['PRODUCTID'])
#         for item in list(data[data['NUMBERITEM']< 0].PRODUCTID):
#             data.loc[data.PRODUCTID == item, 'NUMBERITEM'] = basket_customer_session[basket_customer_session['PRODUCTID']==item].NUMBERITEM.sum()
#         record = data[data['NUMBERITEM']>0]
#         if record.shape[0] > 0:
#                 list_order_id.append(list(record.PRODUCTID.unique()))


In [122]:
len(list_order_id)

5

In [132]:
basket_market_one_user.to_csv('test.csv')

In [135]:
list_order_id

[[85214, 102112, 82126, 86158, 108371, 96459, 77107],
 [104540],
 [104540, 102111],
 [80602,
  79279,
  83369,
  79269,
  76527,
  93557,
  104299,
  79311,
  96459,
  85803,
  81994,
  105838,
  77291,
  104950,
  85214,
  86158,
  82186,
  77107],
 [80602,
  83369,
  93557,
  76527,
  79279,
  79269,
  104299,
  79311,
  96459,
  85803,
  81994,
  105838,
  77291,
  104950,
  86158,
  85214,
  82186,
  77107]]

In [127]:
list_order_id[-2]

[80602,
 79279,
 83369,
 79269,
 76527,
 93557,
 104299,
 79311,
 96459,
 85803,
 81994,
 105838,
 77291,
 104950,
 85214,
 86158,
 82186,
 77107]

In [744]:
a = basket_customer[basket_customer['COOKIE']== 'a11ef7ae-c0b6-49c6-ac1b-5d12816a1240']

In [745]:
a.to_csv('test.csv')

In [684]:
l = list()
for sess in a.SESSION.unique():
    l.append((sess, len(a[a['SESSION']==sess])))

In [685]:
l

[(u'sgsjwxlbns9icz72tjhl9 ', 13),
 (u'1h88bkzy44v4svyp7gpv2h', 2),
 (u'ghk9663onp1jgsxqgurop ', 19),
 (u'0t8nazfzzodjs0nen1otfb', 18),
 (u'c6adno922j6vrtwr964je8', 26),
 (u'94uhnzstpeivnsdny0yih ', 24),
 (u'440n4pqxnhtzya1wt8vxa ', 6),
 (u'3vr9utax326u01vwevisjg', 3),
 (u'btr0ai8r6vsjm6qf5wc3qo', 149),
 (u'6d7gygd36l6tieuek8i2lc', 62),
 (u'lrahrpbb5reo3ovlyyuvh ', 8),
 (u'cpq9fb5ckcdgkwwxdxl6  ', 10),
 (u'etnrtvofjdvx0uemlm5y6m', 8),
 (u'nb1p04tovx4bqoxzl5uru ', 13),
 (u'9eydw5432kf7fsmnkw9aq2', 3),
 (u'aby4emd6nadnfwpdsu5mn ', 20),
 (u'f66oux2hdu0dblj0ss3fe6', 8),
 (u'l8f4zyruaahnxckdt52u  ', 17),
 (u'ljamfx888rvz18uspqc7l ', 1),
 (u'rgtkrb61l1nqvxib82ina ', 1),
 (u'd1zwv0ugvlm5pqxw9tdxwl', 1),
 (u'b82farhjwwpi16rekrhyxb', 6),
 (u'890imtfnvsjh4zdv26m9ul', 3),
 (u'qnu0tnzmvyi9xwl90vwaot', 5),
 (u'8spkfxen6oy4vaz7dulvgt', 20),
 (u'eb2s3k52vnrn24tpt3tk58', 3),
 (u'jv4atdheqcolfmza3tt0ec', 9),
 (u'23dr8j6j8f9ojkwos7eijt', 10),
 (u'gn726vlo7n98ab58y9vduw', 12),
 (u'lkj4qps67zupvw55oz5ml ',

In [516]:
for cookie in basket_customer.COOKIE.unique():
    if rc.exists(cookie):
        rc.delete(cookie)

In [526]:
start = time.time()
list_tmp = get_list_recommend_by_user(basket_customer)
time.time() - start

29.247623920440674

In [ ]:
list_tmp

In [532]:
len(basket_customer.COOKIE.unique())

48912

In [568]:
tmp = basket_customer.pivot_table('ID', index= 'COOKIE', aggfunc = 'count') > 1

In [622]:
tmp = basket_customer.groupby(['COOKIE']).agg('count')

In [624]:
index_ = tmp[tmp['ID']> 1].index.values


In [627]:
len(index_)

27366

In [615]:
tmp

In [191]:
time_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
time_list = np.array(time_list)

time_list[np.arange(0,7)]


# time_list[np.arange(5,7)]

array([1, 2, 3, 4, 5, 6, 7])

In [187]:
a = np.arange(5,6)

In [193]:
time_list[:]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [106]:
 get_cate_ID_by_proID_position(raw.A[0], 0, es)

2515

In [57]:
def get_cate_ID_by_proID_position(productID, position, es):
    try:
        #connect to our cluster
#         from elasticsearch import Elasticsearch
#         es = Elasticsearch([{'host': '172.16.3.105', 'port': 9200}])
        # res = requests.get('http://172.16.3.105:9200/index_product/product/86734')
        # print(res.content)
        pathCate = es.get(index="index_product", doc_type="product", id=str(productID))['_source']['categoryPath']
        listCate = pathCate.split(" ")
        return int(listCate[position])
    except Exception as error:
        log.exception(str(error))   
        return np.nan
    
    
def get_cate_ID_by_proID(productID):
    try:
        pathCate = es.get(index="index_product", doc_type="product", id=str(productID))['_source']['categoryPath']
        listCate = pathCate.split(" ")
        return int(listCate[-1])
    except Exception as error:
        return np.nan    

In [87]:
get_cate_ID_by_proID(76330)
# df['CateID'] = df.ProductID.apply(get_cate_ID_by_proID, args=(0,es,))


2446

In [88]:
get_cate_ID_by_proID_position(76330,0,es)

2515

In [12]:
tmpsaleorderDF['PhoneNumber'] = tmpsaleorderDF['SaleOrderID'].str[-9:]

NameError: name 'tmpsaleorderDF' is not defined

In [ ]:
tmpsaleorderDF.head(1)

In [197]:
import redis
from rediscluster import StrictRedisCluster
# Requires at least one node for cluster discovery. Multiple nodes is recommended.
startup_nodes = [{"host": "10.1.12.30", "port": "16379"}]
rc = StrictRedisCluster(startup_nodes=startup_nodes, decode_responses=True)

In [207]:
rc.exists('tmp')

True

In [213]:
if rc.exists('RecomSys_tmp'):
    rc.delete('RecomSys_tmp')

In [214]:
# get hash
# rc.hget('RecomSys', [2002])
a = rc.hgetall('tmp')

In [216]:
rc.type('tmp')

u'hash'

In [ ]:
rc.hmset()

In [211]:
len(a)

3

In [201]:
rc.expire('RecomSys_tmp', 3)

True

In [ ]:
rc.rename()

In [43]:
today = datetime.date.today()
today = today.strftime('%d/%m/%Y')
command = 'cmd.exe /c echo start C:\Release\\webservices.exe '+today+' '+ today +'> C:\Release\\runme.bat && C:\Release\\runme.bat'


In [44]:
command

'cmd.exe /c echo start C:\\Release\\webservices.exe 16/06/2017 16/06/2017> C:\\Release\\runme.bat && C:\\Release\\runme.bat'

In [34]:
# import functools
# import paramiko
# import socket
# import os
# from stat import S_ISDIR

# class AllowAnythingPolicy(paramiko.MissingHostKeyPolicy):
#     def missing_host_key(self, client, hostname, key):
#         return
# # host = '10.1.12.111'
# # username = 'tgdd'
# # pw = 'Tgdd2012'
# client = paramiko.SSHClient()
# client.set_missing_host_key_policy(AllowAnythingPolicy())
# # client.connect('10.1.12.111', username='tgdd', password = 'Tgdd2012', port = 8897)  # password='')
# client.connect('10.1.4.60', username='tgdd', password = 'Tgdd2012')  # password='')
# def sftp_walk(remotepath):
#     # Kindof a stripped down  version of os.walk, implemented for 
#     # sftp.  Tried running it flat without the yields, but it really
#     # chokes on big directories.
#     path=remotepath
#     files=[]
#     folders=[]
#     for f in sftp.listdir_attr(remotepath):
#         if S_ISDIR(f.st_mode):
#             folders.append(f.filename)
#         else:
#             files.append(f.filename)
#     print (path,folders,files)
#     yield path,folders,files
#     for folder in folders:
#         new_path=os.path.join(remotepath,folder)
#         for x in sftp_walk(new_path):
#             yield x
            
# def get(remotefile,localfile):
#     #  Copy remotefile to localfile, overwriting or creating as needed.
#     sftp.get(remotefile,localfile)
            
# def get_all(remotepath,localpath):
#     #  recursively download a full directory
#     #  Harder than it sounded at first, since paramiko won't walk
#     #
#     # For the record, something like this would gennerally be faster:
#     # ssh user@host 'tar -cz /source/folder' | tar -xz

#     sftp.chdir(os.path.split(remotepath)[0])
#     parent=os.path.split(remotepath)[1]
#     try:
#         os.mkdir(localpath)
#     except:
#         pass
#     for walker in sftp_walk(parent):
#         try:
#             os.mkdir(os.path.join(localpath,walker[0]))
#         except:
#             pass
#         for file in walker[2]:
#             get(os.path.join(walker[0],file),os.path.join(localpath,walker[0],file))

# sftp = client.open_sftp()
# remote_machine = './TA/cropall'
# origin = '/home/tgdd/TA_69/test'

# get_all(remote_machine, origin)# copy file from origin to remote machine

# client.close()

In [ ]:
datetime.datetime.today().strftime('%Y%m%d%H')

In [60]:
datetime.datetime.today().strftime('%d/%m/%Y - %H/%M/%S')

'28/06/2017 - 14/37/16'

In [35]:
rc.hset('tmp', 'key', 'value')

1L

In [52]:
rc.h('tmp', 'key1', 'value1')

0L

In [196]:
rc.hgetall('tmp')

NameError: name 'rc' is not defined

In [47]:
src = u'tmp'
dst = u'abc'
src = src.encode('utf-8').strip()
dst = dst.encode('utf-8').strip()
rc.rename(src, dst)


UnicodeDecodeError: 'utf8' codec can't decode byte 0xff in position 24: invalid start byte